In [2]:
import dapla as dp
import pandas as pd

In [3]:
# Se alle "mock"-datasettene i /felles/ på dapla
dp.show('/felles/mock')

,path,timestamp
0,/felles/mock_sysselsatte/companies_2019_10000,2021-02-23 09:44:16.678
1,/felles/mock_sysselsatte/companies_2019_30000,2021-03-03 07:05:35.853
2,/felles/mock_sysselsatte/companies_2020_10000,2021-02-26 16:00:24.381
3,/felles/mock_sysselsatte/companies_2020_30000,2021-03-03 11:00:33.237
4,/felles/mock_sysselsatte/companies_2021_10000,2021-02-26 16:01:25.697
5,/felles/mock_sysselsatte/companies_2021_30000,2021-03-03 11:05:14.427
6,/felles/mock_sysselsatte/companies_2022_10000,2021-02-26 16:02:25.987
7,/felles/mock_sysselsatte/companies_2022_30000,2021-03-03 11:10:23.890
8,/felles/mock_sysselsatte/companies_2023_10000,2021-02-26 16:03:14.080
9,/felles/mock_sysselsatte/companies_2023_30000,2021-03-03 11:14:50.105


In [4]:
# Stapp datasettene som "objekter" inn i ett "variabelnavn"
comp = dp.read_pandas('/felles/mock_sysselsatte/companies_2019_30000')
pop = dp.read_pandas('/felles/mock_sysselsatte/population_2019_30000')

In [5]:
#Print preview av ett datasett
comp

,work_id,nace,region_code,region,employee_points
0,118415099,11.070,03,Oslo,60
1,165189001,10.920,03,Oslo,54
2,205912967,10.130,02,Akershus (-2019),48
3,959013544,46.693,12,Hordaland (-2019),46
4,405013223,09.900,02,Akershus (-2019),88
...,...,...,...,...,...
1128,706734220,51.210,50,Trøndelag - Trööndelage,15
1129,156404021,74.200,18,Nordland,23
1130,962214437,74.102,18,Nordland,12
1131,999083400,43.341,10,Vest-Agder (-2019),21


In [6]:
# Og det andre
pop

,id,work_id,sex,age,year_birth,work_percent
0,29101946112,None,Menn,0,2019,NaN
1,26101930288,None,Menn,0,2019,NaN
2,22091952214,None,Menn,0,2019,NaN
3,04071997222,None,Menn,0,2019,NaN
4,07101970642,None,Menn,0,2019,NaN
...,...,...,...,...,...,...
29995,15072660901,None,Kvinner,93,1926,NaN
29996,23012604128,None,Kvinner,93,1926,NaN
29997,14102931680,None,Kvinner,90,1929,NaN
29998,29062932768,None,Kvinner,90,1929,NaN


### Døp om variabler innad i kolonner, er dette "omkoding"?

In [21]:
# En dict av dicts, kolonnenavn: verdi - fra:til
omkodingskatalog = {
    'sex' : {
        'Menn' : 1,
        'Kvinner' : 2
    },
    'region' : {
        'Akershus (-2019)' : 'Akershus',
        'Hordaland (-2019)' : 'Hordaland',
        'Trøndelag - Trööndelage' : 'Trøndelag'
    }
}

# En av dictene i dicten
print(omkodingskatalog['sex'])
print(type(omkodingskatalog['sex']))

{'Menn': 1, 'Kvinner': 2}
<class 'dict'>


In [17]:
pop.replace(omkodingskatalog)

,id,work_id,sex,age,year_birth,work_percent
0,29101946112,None,1,0,2019,NaN
1,26101930288,None,1,0,2019,NaN
2,22091952214,None,1,0,2019,NaN
3,04071997222,None,1,0,2019,NaN
4,07101970642,None,1,0,2019,NaN
...,...,...,...,...,...,...
29995,15072660901,None,2,93,1926,NaN
29996,23012604128,None,2,93,1926,NaN
29997,14102931680,None,2,90,1929,NaN
29998,29062932768,None,2,90,1929,NaN


In [18]:
comp.replace(omkodingskatalog)

,work_id,nace,region_code,region,employee_points
0,118415099,11.070,03,Oslo,60
1,165189001,10.920,03,Oslo,54
2,205912967,10.130,02,Akershus,48
3,959013544,46.693,12,Hordaland,46
4,405013223,09.900,02,Akershus,88
...,...,...,...,...,...
1128,706734220,51.210,50,Trøndelag,15
1129,156404021,74.200,18,Nordland,23
1130,962214437,74.102,18,Nordland,12
1131,999083400,43.341,10,Vest-Agder (-2019),21


### Opprett en ny kolonne med "grupperingsvariabler", avhengig av en "katalog" fra KLASS

Landsinndeling
https://www.ssb.no/klass/klassifikasjoner/106

KLASS-API
https://data.ssb.no/api/klass/v1/api-guide.html

In [40]:
import json
import requests
url = 'http://data.ssb.no/api/klass/v1/correspondencetables/468'
headers = {'Accept': 'application/json'}
#Gjoer etterspoersel
r = requests.get(url, headers = headers)
#Av en eller annen grunn returnerer KLASS APIen json som bytestring...
#La oss decode...
cont = r.content.decode('utf8').replace("'", '"')
#Konverter til en liste vi kan bruke
cont = json.loads(cont)
cont = cont['correspondenceMaps']

In [47]:
# Det vi har fått fra KLASS er nå en liste med dicts
cont

[{'sourceCode': '1',
  'sourceName': 'Oslo og Viken',
  'targetCode': '03',
  'targetName': 'Oslo'},
 {'sourceCode': '1',
  'sourceName': 'Oslo og Viken',
  'targetCode': '30',
  'targetName': 'Viken'},
 {'sourceCode': '2',
  'sourceName': 'Innlandet',
  'targetCode': '34',
  'targetName': 'Innlandet'},
 {'sourceCode': '3',
  'sourceName': 'Agder og Sør-Østlandet',
  'targetCode': '38',
  'targetName': 'Vestfold og Telemark'},
 {'sourceCode': '3',
  'sourceName': 'Agder og Sør-Østlandet',
  'targetCode': '42',
  'targetName': 'Agder'},
 {'sourceCode': '4',
  'sourceName': 'Vestlandet',
  'targetCode': '11',
  'targetName': 'Rogaland'},
 {'sourceCode': '4',
  'sourceName': 'Vestlandet',
  'targetCode': '15',
  'targetName': 'Møre og Romsdal'},
 {'sourceCode': '4',
  'sourceName': 'Vestlandet',
  'targetCode': '46',
  'targetName': 'Vestland'},
 {'sourceCode': '5',
  'sourceName': 'Trøndelag',
  'targetCode': '50',
  'targetName': 'Trøndelag - Trööndelage'},
 {'sourceCode': '6',
  'sourc

In [52]:
# Loop over katalogen (så vi har tilgang på hver dict i listen)
for corr in cont:
    # En slags progress-rapport?
    print(corr['targetCode'], corr['sourceCode'])
    # Lag en ny kolonne for å holde på landsdel_koden, sett denne til sourceCode i katalogen, 
    # der region-koden matcher på targetCode.
    comp.loc[comp['region_code'] == corr['targetCode'], 'landsdel_kode' ] = corr['sourceCode']
    # Samme, bare for navnet på landsdelen
    comp.loc[comp['region_code'] == corr['targetCode'], 'landsdel_navn' ] = corr['sourceName']

03 1
30 1
34 2
38 3
42 3
11 4
15 4
46 4
50 5
18 6
54 6


In [58]:
comp[['region', 'landsdel_navn']].sample(10)

,region,landsdel_navn
280,Møre og Romsdal,Vestlandet
677,Nordland,Nord-Norge
991,Vest-Agder (-2019),NaN
156,Hordaland (-2019),NaN
218,Akershus (-2019),NaN
557,Nordland,Nord-Norge
292,Oslo,Oslo og Viken
128,Vest-Agder (-2019),NaN
199,Oppland (-2019),NaN
528,Troms - Romsa (-2019),NaN


In [54]:
#Unike fylker som fortsatt mangler landsinndeling
pd.unique(comp[comp['landsdel_kode'].isna()]['region'])

array(['Akershus (-2019)', 'Hordaland (-2019)', 'Østfold (-2019)',
       'Buskerud (-2019)', 'Vestfold (-2019)', 'Troms - Romsa (-2019)',
       'Vest-Agder (-2019)', 'Hedmark (-2019)', 'Oppland (-2019)',
       'Telemark (-2019)', 'Sogn og Fjordane (-2019)',
       'Aust-Agder (-2019)', 'Finnmark - Finnmárku (-2019)', None],
      dtype=object)

In [55]:
# Da må man enten gå gjennom å knytte disse gamle fylkene til landsdeler, eller finne flere korrespondansetabeller
pd.unique(comp[comp['landsdel_kode'].isna()]['region_code'])

array(['02', '12', '01', '06', '07', '19', '10', '04', '05', '08', '14',
       '09', '20', None], dtype=object)